# SGLang Frontend Language

SGLang frontend language can be used to define simple and easy prompts in a convenient, structured way.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
import requests
import os

from sglang import assistant_begin, assistant_end
from sglang import assistant, function, gen, system, user
from sglang import image
from sglang import RuntimeEndpoint, set_default_backend
from sglang.srt.utils import load_image
from sglang.test.test_utils import is_in_ci
from sglang.utils import print_highlight, terminate_process, wait_for_server

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-07-10 16:27:29] server_args=ServerArgs(model_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=33872, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=835949815, constrained_json_whitespace_pattern

[2025-07-10 16:27:41] Attention backend not set. Use fa3 backend by default.
[2025-07-10 16:27:41] Init torch distributed begin.


[2025-07-10 16:27:41] Init torch distributed ends. mem usage=0.00 GB


[2025-07-10 16:27:42] Load weight begin. avail mem=78.50 GB
[2025-07-10 16:27:42] The weight of LmHead is not packed


[2025-07-10 16:27:43] Using model weights format ['*.safetensors']


...

[2025-07-10 16:27:48] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=62.81 GB


[2025-07-10 16:27:48] INFO:     Started server process [2779242]
[2025-07-10 16:27:48] INFO:     Waiting for application startup.
[2025-07-10 16:27:48] INFO:     Application startup complete.
[2025-07-10 16:27:48] INFO:     Uvicorn running on http://0.0.0.0:33872 (Press CTRL+C to quit)
[2025-07-10 16:27:48] INFO:     127.0.0.1:47184 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-10 16:27:49] INFO:     127.0.0.1:47190 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-10 16:27:49] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-10T16:27:49.791223


[2025-07-10 16:27:50] INFO:     127.0.0.1:47202 - "POST /generate HTTP/1.1" 200 OK
[2025-07-10 16:27:50] The server is fired up and ready to roll!


Server started on http://localhost:33872


Set the default backend. Note: Besides the local server, you may use also `OpenAI` or other API endpoints.

In [2]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-07-10 16:27:53] INFO:     127.0.0.1:40920 - "GET /get_model_info HTTP/1.1" 200 OK


## Basic Usage

The most simple way of using SGLang frontend language is a simple question answer dialog between a user and an assistant.

In [3]:
@function
def basic_qa(s, question):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user(question)
    s += assistant(gen("answer", max_tokens=512))

In [4]:
state = basic_qa("List 3 countries and their capitals.")
print_highlight(state["answer"])

[2025-07-10 16:27:53] Prefill batch. #new-seq: 1, #new-token: 31, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-10T16:27:53.861310


[2025-07-10 16:27:54] INFO:     127.0.0.1:40928 - "POST /generate HTTP/1.1" 200 OK


## Multi-turn Dialog

SGLang frontend language can also be used to define multi-turn dialogs.

In [5]:
@function
def multi_turn_qa(s):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user("Please give me a list of 3 countries and their capitals.")
    s += assistant(gen("first_answer", max_tokens=512))
    s += user("Please give me another list of 3 countries and their capitals.")
    s += assistant(gen("second_answer", max_tokens=512))
    return s


state = multi_turn_qa()
print_highlight(state["first_answer"])
print_highlight(state["second_answer"])

[2025-07-10 16:27:54] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 18, #token: 18, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-10T16:27:54.172838
[2025-07-10 16:27:54] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, cuda graph: False, gen throughput (token/s): 6.46, #queue-req: 0, timestamp: 2025-07-10T16:27:54.233666


[2025-07-10 16:27:54] INFO:     127.0.0.1:40944 - "POST /generate HTTP/1.1" 200 OK


[2025-07-10 16:27:54] Prefill batch. #new-seq: 1, #new-token: 23, #cached-token: 68, #token: 68, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-10T16:27:54.499781
[2025-07-10 16:27:54] Decode batch. #running-req: 1, #token: 102, token usage: 0.00, cuda graph: False, gen throughput (token/s): 105.80, #queue-req: 0, timestamp: 2025-07-10T16:27:54.611722


[2025-07-10 16:27:54] INFO:     127.0.0.1:40952 - "POST /generate HTTP/1.1" 200 OK


## Control flow

You may use any Python code within the function to define more complex control flows.

In [6]:
@function
def tool_use(s, question):
    s += assistant(
        "To answer this question: "
        + question
        + ". I need to use a "
        + gen("tool", choices=["calculator", "search engine"])
        + ". "
    )

    if s["tool"] == "calculator":
        s += assistant("The math expression is: " + gen("expression"))
    elif s["tool"] == "search engine":
        s += assistant("The key word to search is: " + gen("word"))


state = tool_use("What is 2 * 2?")
print_highlight(state["tool"])
print_highlight(state["expression"])

[2025-07-10 16:27:54] Prefill batch. #new-seq: 1, #new-token: 25, #cached-token: 8, #token: 8, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-10T16:27:54.807051
[2025-07-10 16:27:54] INFO:     127.0.0.1:40960 - "POST /generate HTTP/1.1" 200 OK
[2025-07-10 16:27:54] Prefill batch. #new-seq: 1, #new-token: 2, #cached-token: 31, #token: 31, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-10T16:27:54.832068
[2025-07-10 16:27:54] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 31, #token: 33, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-10T16:27:54.832955
[2025-07-10 16:27:54] INFO:     127.0.0.1:40962 - "POST /generate HTTP/1.1" 200 OK


[2025-07-10 16:27:54] Prefill batch. #new-seq: 1, #new-token: 13, #cached-token: 33, #token: 33, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-10T16:27:54.868089


[2025-07-10 16:27:55] Decode batch. #running-req: 1, #token: 63, token usage: 0.00, cuda graph: False, gen throughput (token/s): 93.35, #queue-req: 0, timestamp: 2025-07-10T16:27:55.040220


[2025-07-10 16:27:55] Decode batch. #running-req: 1, #token: 103, token usage: 0.01, cuda graph: False, gen throughput (token/s): 110.94, #queue-req: 0, timestamp: 2025-07-10T16:27:55.400763
[2025-07-10 16:27:55] INFO:     127.0.0.1:40978 - "POST /generate HTTP/1.1" 200 OK


## Parallelism

Use `fork` to launch parallel prompts. Because `sgl.gen` is non-blocking, the for loop below issues two generation calls in parallel.

In [7]:
@function
def tip_suggestion(s):
    s += assistant(
        "Here are two tips for staying healthy: "
        "1. Balanced Diet. 2. Regular Exercise.\n\n"
    )

    forks = s.fork(2)
    for i, f in enumerate(forks):
        f += assistant(
            f"Now, expand tip {i+1} into a paragraph:\n"
            + gen("detailed_tip", max_tokens=256, stop="\n\n")
        )

    s += assistant("Tip 1:" + forks[0]["detailed_tip"] + "\n")
    s += assistant("Tip 2:" + forks[1]["detailed_tip"] + "\n")
    s += assistant(
        "To summarize the above two tips, I can say:\n" + gen("summary", max_tokens=512)
    )


state = tip_suggestion()
print_highlight(state["summary"])

[2025-07-10 16:27:55] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, #token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-10T16:27:55.519527
[2025-07-10 16:27:55] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, #token: 49, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-10T16:27:55.520486


[2025-07-10 16:27:55] Decode batch. #running-req: 2, #token: 110, token usage: 0.01, cuda graph: False, gen throughput (token/s): 122.87, #queue-req: 0, timestamp: 2025-07-10T16:27:55.946047


[2025-07-10 16:27:56] Decode batch. #running-req: 2, #token: 190, token usage: 0.01, cuda graph: False, gen throughput (token/s): 211.38, #queue-req: 0, timestamp: 2025-07-10T16:27:56.324508
[2025-07-10 16:27:56] INFO:     127.0.0.1:40986 - "POST /generate HTTP/1.1" 200 OK


[2025-07-10 16:27:56] Decode batch. #running-req: 1, #token: 157, token usage: 0.01, cuda graph: False, gen throughput (token/s): 162.07, #queue-req: 0, timestamp: 2025-07-10T16:27:56.694724


[2025-07-10 16:27:56] INFO:     127.0.0.1:40998 - "POST /generate HTTP/1.1" 200 OK
[2025-07-10 16:27:56] Prefill batch. #new-seq: 1, #new-token: 253, #cached-token: 39, #token: 39, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-10T16:27:56.980968
[2025-07-10 16:27:57] Decode batch. #running-req: 1, #token: 301, token usage: 0.01, cuda graph: False, gen throughput (token/s): 101.03, #queue-req: 0, timestamp: 2025-07-10T16:27:57.090644


[2025-07-10 16:27:57] Decode batch. #running-req: 1, #token: 341, token usage: 0.02, cuda graph: False, gen throughput (token/s): 110.49, #queue-req: 0, timestamp: 2025-07-10T16:27:57.452676


[2025-07-10 16:27:57] Decode batch. #running-req: 1, #token: 381, token usage: 0.02, cuda graph: False, gen throughput (token/s): 110.35, #queue-req: 0, timestamp: 2025-07-10T16:27:57.815174


[2025-07-10 16:27:58] Decode batch. #running-req: 1, #token: 421, token usage: 0.02, cuda graph: False, gen throughput (token/s): 110.52, #queue-req: 0, timestamp: 2025-07-10T16:27:58.177091


[2025-07-10 16:27:58] Decode batch. #running-req: 1, #token: 461, token usage: 0.02, cuda graph: False, gen throughput (token/s): 110.36, #queue-req: 0, timestamp: 2025-07-10T16:27:58.539539


[2025-07-10 16:27:58] INFO:     127.0.0.1:41000 - "POST /generate HTTP/1.1" 200 OK


## Constrained Decoding

Use `regex` to specify a regular expression as a decoding constraint. This is only supported for local models.

In [8]:
@function
def regular_expression_gen(s):
    s += user("What is the IP address of the Google DNS servers?")
    s += assistant(
        gen(
            "answer",
            temperature=0,
            regex=r"((25[0-5]|2[0-4]\d|[01]?\d\d?).){3}(25[0-5]|2[0-4]\d|[01]?\d\d?)",
        )
    )


state = regular_expression_gen()
print_highlight(state["answer"])

[2025-07-10 16:27:58] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 12, #token: 12, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-10T16:27:58.856939


[2025-07-10 16:27:59] Decode batch. #running-req: 1, #token: 0, token usage: 0.00, cuda graph: False, gen throughput (token/s): 34.73, #queue-req: 0, timestamp: 2025-07-10T16:27:59.691141
[2025-07-10 16:27:59] INFO:     127.0.0.1:41012 - "POST /generate HTTP/1.1" 200 OK


Use `regex` to define a `JSON` decoding schema.

In [9]:
character_regex = (
    r"""\{\n"""
    + r"""    "name": "[\w\d\s]{1,16}",\n"""
    + r"""    "house": "(Gryffindor|Slytherin|Ravenclaw|Hufflepuff)",\n"""
    + r"""    "blood status": "(Pure-blood|Half-blood|Muggle-born)",\n"""
    + r"""    "occupation": "(student|teacher|auror|ministry of magic|death eater|order of the phoenix)",\n"""
    + r"""    "wand": \{\n"""
    + r"""        "wood": "[\w\d\s]{1,16}",\n"""
    + r"""        "core": "[\w\d\s]{1,16}",\n"""
    + r"""        "length": [0-9]{1,2}\.[0-9]{0,2}\n"""
    + r"""    \},\n"""
    + r"""    "alive": "(Alive|Deceased)",\n"""
    + r"""    "patronus": "[\w\d\s]{1,16}",\n"""
    + r"""    "bogart": "[\w\d\s]{1,16}"\n"""
    + r"""\}"""
)


@function
def character_gen(s, name):
    s += user(
        f"{name} is a character in Harry Potter. Please fill in the following information about this character."
    )
    s += assistant(gen("json_output", max_tokens=256, regex=character_regex))


state = character_gen("Harry Potter")
print_highlight(state["json_output"])

[2025-07-10 16:28:00] Prefill batch. #new-seq: 1, #new-token: 24, #cached-token: 14, #token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-10T16:28:00.171725


[2025-07-10 16:28:00] Decode batch. #running-req: 1, #token: 79, token usage: 0.00, cuda graph: False, gen throughput (token/s): 45.16, #queue-req: 0, timestamp: 2025-07-10T16:28:00.576879


[2025-07-10 16:28:00] Decode batch. #running-req: 1, #token: 119, token usage: 0.01, cuda graph: False, gen throughput (token/s): 100.60, #queue-req: 0, timestamp: 2025-07-10T16:28:00.974493


[2025-07-10 16:28:01] Decode batch. #running-req: 1, #token: 159, token usage: 0.01, cuda graph: False, gen throughput (token/s): 105.71, #queue-req: 0, timestamp: 2025-07-10T16:28:01.352866
[2025-07-10 16:28:01] INFO:     127.0.0.1:41016 - "POST /generate HTTP/1.1" 200 OK


## Batching 

Use `run_batch` to run a batch of prompts.

In [10]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


states = text_qa.run_batch(
    [
        {"question": "What is the capital of the United Kingdom?"},
        {"question": "What is the capital of France?"},
        {"question": "What is the capital of Japan?"},
    ],
    progress_bar=True,
)

for i, state in enumerate(states):
    print_highlight(f"Answer {i+1}: {states[i]['answer']}")

[2025-07-10 16:28:01] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 13, #token: 13, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-10T16:28:01.464558
[2025-07-10 16:28:01] INFO:     127.0.0.1:41030 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/3 [00:00<?, ?it/s]

[2025-07-10 16:28:01] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 17, #token: 17, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-10T16:28:01.488538
[2025-07-10 16:28:01] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 17, #token: 28, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-10T16:28:01.489591
[2025-07-10 16:28:01] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 19, #token: 37, token usage: 0.00, #running-req: 2, #queue-req: 0, timestamp: 2025-07-10T16:28:01.517824


 33%|███▎      | 1/3 [00:00<00:00,  8.27it/s]

 67%|██████▋   | 2/3 [00:00<00:00, 13.12it/s]

100%|██████████| 3/3 [00:00<00:00, 21.48it/s]

[2025-07-10 16:28:01] INFO:     127.0.0.1:41050 - "POST /generate HTTP/1.1" 200 OK
[2025-07-10 16:28:01] INFO:     127.0.0.1:41056 - "POST /generate HTTP/1.1" 200 OK
[2025-07-10 16:28:01] INFO:     127.0.0.1:41036 - "POST /generate HTTP/1.1" 200 OK


## Streaming 

Use `stream` to stream the output to the user.

In [11]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


state = text_qa.run(
    question="What is the capital of France?", temperature=0.1, stream=True
)

for out in state.text_iter():
    print(out, end="", flush=True)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant


[2025-07-10 16:28:01] INFO:     127.0.0.1:41062 - "POST /generate HTTP/1.1" 200 OK
[2025-07-10 16:28:01] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 25, #token: 25, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-10T16:28:01.635278
The

 capital

 of

 France

 is

 Paris

.

<|im_end|>


## Complex Prompts

You may use `{system|user|assistant}_{begin|end}` to define complex prompts.

In [12]:
@function
def chat_example(s):
    s += system("You are a helpful assistant.")
    # Same as: s += s.system("You are a helpful assistant.")

    with s.user():
        s += "Question: What is the capital of France?"

    s += assistant_begin()
    s += "Answer: " + gen("answer", max_tokens=100, stop="\n")
    s += assistant_end()


state = chat_example()
print_highlight(state["answer"])

[2025-07-10 16:28:01] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 14, #token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-10T16:28:01.724283


[2025-07-10 16:28:01] INFO:     127.0.0.1:41064 - "POST /generate HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-07-10 16:28:01] Child process unexpectedly failed with exitcode=9. pid=2779579
[2025-07-10 16:28:01] Child process unexpectedly failed with exitcode=9. pid=2779410


## Multi-modal Generation

You may use SGLang frontend language to define multi-modal prompts.
See [here](https://docs.sglang.ai/supported_models/generative_models.html) for supported models.

In [14]:
server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-VL-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-07-10 16:28:13] server_args=ServerArgs(model_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-VL-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=32052, nccl_port=None, mem_fraction_static=0.7866, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=876773949, constrained_json_whitespa

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
[2025-07-10 16:28:15] You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


[2025-07-10 16:28:16] Inferred chat template from model path: qwen2-vl


You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
[2025-07-10 16:28:25] You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


[2025-07-10 16:28:25] Attention backend not set. Use flashinfer backend by default.
[2025-07-10 16:28:25] Init torch distributed begin.
[2025-07-10 16:28:25] Init torch distributed ends. mem usage=0.00 GB


...

[2025-07-10 16:28:33] INFO:     Started server process [2780380]
[2025-07-10 16:28:33] INFO:     Waiting for application startup.
[2025-07-10 16:28:33] INFO:     Application startup complete.
[2025-07-10 16:28:33] INFO:     Uvicorn running on http://0.0.0.0:32052 (Press CTRL+C to quit)


[2025-07-10 16:28:33] INFO:     127.0.0.1:46112 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-10 16:28:34] INFO:     127.0.0.1:46116 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-10 16:28:34] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-10T16:28:34.403774


[2025-07-10 16:28:35] INFO:     127.0.0.1:46120 - "POST /generate HTTP/1.1" 200 OK
[2025-07-10 16:28:35] The server is fired up and ready to roll!


Server started on http://localhost:32052


In [15]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-07-10 16:28:38] INFO:     127.0.0.1:46130 - "GET /get_model_info HTTP/1.1" 200 OK


Ask a question about an image.

In [16]:
@function
def image_qa(s, image_file, question):
    s += user(image(image_file) + question)
    s += assistant(gen("answer", max_tokens=256))


image_url = "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
image_bytes, _ = load_image(image_url)
state = image_qa(image_bytes, "What is in the image?")
print_highlight(state["answer"])

[2025-07-10 16:28:39] Prefill batch. #new-seq: 1, #new-token: 307, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-10T16:28:39.936735


[2025-07-10 16:28:40] Decode batch. #running-req: 1, #token: 340, token usage: 0.02, cuda graph: False, gen throughput (token/s): 5.13, #queue-req: 0, timestamp: 2025-07-10T16:28:40.736048


[2025-07-10 16:28:41] Decode batch. #running-req: 1, #token: 380, token usage: 0.02, cuda graph: False, gen throughput (token/s): 60.93, #queue-req: 0, timestamp: 2025-07-10T16:28:41.392525


[2025-07-10 16:28:41] INFO:     127.0.0.1:46138 - "POST /generate HTTP/1.1" 200 OK


In [17]:
terminate_process(server_process)